# Trading agent demo

## Imports

In [1]:
import os
import json
import torch
import numpy as np

from modules.data_loader import DataLoader
from modules.single_asset_env import SingleAssetEnv
from modules.q_network import Q_network
from modules.memory import Memory
from modules.trading_agent import TradingAgent
from modules.functions import train_agent, test_agent

## Parameters and variables

In [2]:
num_actions = int(os.environ["NUM_ACTIONS"])
states_dim = int(os.environ["STATES_DIM"])

num_epochs = int(os.environ["NUM_EPOCHS"])
batch_size = int(os.environ["BATCH_SIZE"])
memory_size = int(os.environ["MEMORY_SIZE"])

learning_rate = float(os.environ["LEARNING_RATE"])
learning_freq = int(os.environ["LEARNING_FREQ"])

tau = float(os.environ["TAU"])
gamma = float(os.environ["GAMMA"])

device = os.environ["DEVICE"]
seed = int(os.environ["SEED"])

## Loading data

In [3]:
dataloader = DataLoader(os.environ["START_DATE"], 
                        os.environ["END_DATE"],
                        os.environ["FREQ"],
                        os.environ["TRAIN_TEST_SPLIT_DATE"])

assets = json.loads(os.environ["ASSETS"])

trains = []
tests = []

for asset in assets:
    train, test = dataloader.load(asset)

    trains.append(train)
    tests.append(test)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


## Initialisations

In [4]:
train_envs = [SingleAssetEnv(train) for train in trains]

agent = TradingAgent(states_dim, num_actions, assets, 
                     batch_size=batch_size,
                     memory_size=memory_size,
                     learning_rate=learning_rate,
                     tau=tau,
                     gamma=gamma,
                     learning_freq=learning_freq,
                     device=device,
                     seed=seed)

## Training the agent

In [5]:
train_agent(assets, agent, train_envs, num_epochs)

Epoch  0 | Scores = {'BTC-USD': -9112006.48828125, 'ETH-USD': 499145168.0540314, 'BNB-USD': 15569141.860240936}
Epoch  1 | Scores = {'BTC-USD': 1872995.271484375, 'ETH-USD': 509770981.7358246, 'BNB-USD': 52754580.03080559}
Epoch  2 | Scores = {'BTC-USD': 17864036.131347656, 'ETH-USD': 509770981.7358246, 'BNB-USD': 52754580.03080559}
Epoch  3 | Scores = {'BTC-USD': -321500.0, 'ETH-USD': 509770981.7358246, 'BNB-USD': 52754580.03080559}
Epoch  4 | Scores = {'BTC-USD': 47335074.458984375, 'ETH-USD': 258236078.15629578, 'BNB-USD': 53062142.75041008}
Epoch  5 | Scores = {'BTC-USD': -121700.0, 'ETH-USD': 509770981.7358246, 'BNB-USD': 52754580.03080559}
Epoch  6 | Scores = {'BTC-USD': -421400.0, 'ETH-USD': 509770981.7358246, 'BNB-USD': 52754580.03080559}
Epoch  7 | Scores = {'BTC-USD': 12179509.396972656, 'ETH-USD': 353624095.19750977, 'BNB-USD': 52754580.03080559}
Epoch  8 | Scores = {'BTC-USD': 54107.8984375, 'ETH-USD': 509770981.7358246, 'BNB-USD': 52754580.03080559}
Epoch  9 | Scores = {'B

## Testing the agent

In [8]:
test_envs = [SingleAssetEnv(test) for test in tests]

print('Initial capital : 100000$ for each asset\n')

final_running_balances = test_agent(assets, agent, test_envs)

Initial capital : 100000$ for each asset

Balance : BTC-USD: 183013.55
Balance : ETH-USD: 190956.93
Balance : BNB-USD: 100000.0
----------------------------
Total profit made: 173970.48


In [9]:
# on training data
envs = [SingleAssetEnv(train) for train in trains]

final_running_balances2 = test_agent(assets, agent, envs)

Balance : BTC-USD: 141997.49
Balance : ETH-USD: 632495.76
Balance : BNB-USD: 100000.0
----------------------------
Total profit made: 574493.25
